# Information gain for the linear mixed effects patient outcomes model

All other considerations being equal, which treatment, $x_\mathrm{tx}$, should a patient with biomarkers $x_\mathrm{bm}$ take to best update our understanding of how to predict patient outcomes, $y$?

## Fisher information
If we observed the actual outcome of a treatment, then we could compute a measure of information gain through the Kullback-Liebler divergence (or relative entropy) of the posterior, $P(\theta | y)$, with respect to the prior, $P(\theta)$.

$$ D\big[ P(\theta | y) \ || \ P(\theta) \big] = \int P(\theta | y) \ \log\left(\frac{P(\theta | y)}{P(\theta)}\right) \ \mathrm{d}\theta $$

The relative entropy measures the information lost by approximating $P(\theta | y)$ with $P(\theta)$, and thus it provides a nice way of measuring the usefulness of an experiment after observing the results.  However we'd like a way to quantify the potential usefulness of an experiment prior to observing the results.

The Fisher information (hereafter, just information) is a measure of the maximum extent to which data can update a model.

Formally, if we have a likelihood $f(y|\theta)$ of data, $y$, given model parameters, $\theta$, then the information, $I(\theta)$ provides a lower bound on the variance of the any estimator for the model parameters:

$$ \mathrm{Cov}(\hat{\theta}) \geq I^{-1}(\theta) $$

In the case of a model with more than one parameters, the information is a matrix whose inverse is the lower bound of the parameter covariance matrix.

The information is defined as

$$ I(\theta)_{i,j} = -\int H_\theta \big[ \log f(y | \theta) \big]_{i,j} \ f(y | \theta) \ \mathrm{d}y $$

where $H_\theta$ is the Hessian operator, or the matrix of second derivatives with respect to $\theta$.  As a side note for the geometrically inclined, the information can also be seen as the curvature of the relative entropy.

Notably, the information integrates over all possible results of an experiment, and so we don't need to observe the outcome of an experiment in order to compute its potential to update the model.  We will however, need to put in some assumption for the model parameters, $\theta$.  In a Bayesian inference context, we can simply take the expectation value of $I(\theta)$ over the prior, $P(\theta)$.

## Linear mixed model for patient outcomes

Let's look at the properties of the information for the patient outcomes model.

As a reminder, the model predicts patient outcomes, $y$ (e.g., tumor load or Karnofsky score), given patient covariates, $x_\mathrm{bm}$, a treatment $x_\mathrm{tx}$, and a time after treatment, $t$.

The patient outcome is assumed to be normally distributed with a mean of $\mu = x \cdot \beta + z \cdot u$ and a variance of $\sigma^2_\epsilon$.  

$\beta = (\beta_0, \beta_1, \beta_\mathrm{bm}, \beta_\mathrm{tx}, \beta_\mathrm{int})$ is a vector of model parameters denoting population-level or fixed effects.

$x = (1, t, x_\mathrm{bm}t, x_\mathrm{tx}t, x_\mathrm{int}t)$ is a vector of patient covariates formed from their biomarkers, the chosen treatment, and the time after treatment.

$u = (u_0, u_1)$ is a vector of patient-level or random effects, and $z = (1, t)$ is a vector of patient-level predictors.  The patient-level effects are assumed to be drawn from a normal distribution with a mean of 0 and a covariance matrix,

$$ \Sigma_u = \begin{pmatrix} \sigma^2_0 & \rho \sigma_0 \sigma_1 \\ \rho \sigma_0 \sigma_1 & \sigma^2_1 \end{pmatrix} $$

With some loss of generality but much gain in ease of computation, let's assume that there is no correlation in patient-level effects (that is, $\rho = 0$).  Thus there are three noise terms in the model, $\sigma^2_\epsilon$, $\sigma^2_0$, and $\sigma^2_1$.

Through a little gumption, we can compute the information matrix for parameters $\theta = (\beta, \sigma^2)$ where $\sigma^2 = (\sigma^2_\epsilon, \sigma^2_0, \sigma^2_1)$ as the block matrix

$$ I(\theta) = \begin{pmatrix} I_{\beta\beta} & I_{\beta\sigma^2} \\ I_{\beta\sigma^2}^T & I_{\sigma^2\sigma^2} \end{pmatrix} $$

The population-level effects entries are simply given by

$$ I_{\beta\beta, i, j} = \frac{x_i x_j}{v} $$

where $v = \sigma^2_\epsilon + \sigma^2_0 + \sigma^2_1$.

The cross-terms, in $I_{\beta\sigma^2}$, are conveniently all zero.  The noise block is given by

$$ I_{\sigma^2\sigma^2} = \frac{1}{2v^2} \begin{pmatrix} 1 & t^2 & 1 \\ t^2 & t^4 & t^2 \\ 1 & t^2 & 1 \end{pmatrix} $$

For a generalization of this calculation to non-linear mixed models with exponential family link functions, see [Wand 2007](https://www.sciencedirect.com/science/article/pii/S0047259X07000024).

There are a couple rather obvious (at least in retrospect) takeaways from this calculation.  First, we need to try an experiment with specific patient covariate $x_i$ if we want to get information on the effect size for $\beta_i$.  

Second, the information gained will generally increase with time after treatment.  In other words, if we want to accurately measure a slope in the presence of noise, we should place two data points as far apart from each other as possible.

## Relative information gain

One less obvious takeaway is that the expected value of the information over the prior depends only on the prior distribution of the noise terms, $\sigma^2$, and not the population-level terms, $\beta$.  If we want to incorporate some measure of information gain that is relative to what we already know (our prior), then we could look at the product of the information and the prior covariance.  Let's define the normalized information gain from observing the treatment, $x_\mathrm{tx}$ of a patient with covariates $x_\mathrm{tx}$ after time $t$.

$$ \mathcal{I}(x) = \mathrm{Cov}(\Theta) \cdot \int I(\theta | x) \ P(\theta) \ \mathrm{d}\theta$$ 

where

$$ \mathrm{Cov}(\Theta) = \int (\theta - \mu_\theta) \cdot (\theta - \mu_\theta)^T P(\theta) \ \mathrm{d}\theta $$

is the covariance matrix for the prior distribution, $P(\theta)$, and $\mu_\theta$ is the mean of the prior.

We can use a wide range of invariants of the information matrix as a scalar measure of information gain (see for example the [Wikipedia article on optimal design](https://en.wikipedia.org/wiki/Optimal_design#Minimizing_the_variance_of_estimators)).  The trace, or the sum of the diagonal terms, is one particularly easy to compute measure.  Another scalar invariant measure of this matrix would be the determinant.

## Information gain in action

Let's see what this looks like in a few different scenarios.

First we'll set up a couple convenience functions.

In [39]:
using Plots
using LinearAlgebra: diag, det, tr, dot, eigvals
using Statistics: mean, std, cov
using StatsBase: Histogram, fit, normalize

In [2]:
function pdet(A)
    """
    The pseudodeterminant of A is equal to the product of all non-zero
    eigenvalues.
    """
    λ = eigvals(A)
    return prod(λ[abs.(λ) .>= 2 * eps()])
end

function blockdiag(A...)
    """
    Creates a block diagonal matrix from input matrices in A
    """
    if length(A) == 1
        return A
    end
    N_rows = sum([size(matrix)[1] for matrix in A])
    N_cols = sum([size(matrix)[2] for matrix in A])
    new_matrix = zeros(N_rows, N_cols)
    row_start = 1
    col_start = 1
    for (i, matrix) in enumerate(A)
        rows, cols = size(matrix)
        row_end = row_start + rows - 1
        col_end = col_start + cols - 1
        new_matrix[row_start:row_end, col_start:col_end] = matrix
        row_start = row_end + 1
        col_start = col_end + 1
    end
    return new_matrix
end

function fisher(θ_v, t, x_bm, x_tx)
    """
    Compute the fisher information from
    θ_v : array of total variance (σ2_0 + σ2_1 + σ2_ϵ) prior draws
    t : time after treatment
    x_bm : array of biomarker covariates
    x_tx : array of treatment covariates
    """
    x_int = [bm * tx for bm in x_bm for tx in x_tx]
    N_bm = length(x_bm)
    N_tx = length(x_tx)
    N_int = N_bm * N_tx
    N_parameters = 2 + N_bm + N_tx + N_int + 3
    N_samples = length(θ_v)
    x = cat([1.0, t], x_bm, x_tx, x_int; dims=1)
    I = zeros(N_samples, N_parameters, N_parameters)
    for i in 1:N_samples
        v = θ_v[i]
        I_ββ = x * inv(v) * transpose(x)
        I_σ2σ2 = 0.5 * inv(v) ^ 2 * [1 t^2 1; t^2 t^4 t^2; 1 t^2 1]
        I[i, :, :] = blockdiag(I_ββ, I_σ2σ2)
    end
    if N_samples == 1
        return dropdims(I, dims=1)
    end
    return I
end

fisher (generic function with 1 method)

We'll look at a simple case of two biomarker covariates and two treatments.  We'll use contrast coding for the biomarkers (+1 if present, -1 if not present) and indicator variables (+1 if present, 0 if not present) for the treatments.

For now we're not so worried about the time dynamics so we'll just set $t = 1$.

In [3]:
N_bm = 2
N_tx = 2
N_int = N_bm + N_tx
N_parameters = 2 + N_bm + N_tx + N_int + 3
t = 1;

### Low-information prior

Let's assume a set of wide priors like we might have before seeing many patients.  We'll have a normal prior over effect parameters, centered at 0, with a spread of 30 (arbitrary-ish units).  For the variance terms we'll use a log-normal prior with a mean of 10 and a standard deviation of about 1

In [92]:
mean_β = 0.0
scale_β = 30.0
mean_σ2 = 10.0
scale_σ2 = 0.1

μ_θ = cat(mean_β * ones(N_parameters - 3), mean_σ2 * ones(3), dims=1)
σ_θ = cat(scale_β * ones(N_parameters - 3), scale_σ2 * ones(3), dims=1)

N_samples = 10000

prior = Array{Float64, 2}(undef, N_samples, N_parameters)
for i in 1:N_parameters
    if i < N_parameters - 2
        # normal prior for effect size terms
        prior[:, i] = μ_θ[i] .+ σ_θ[i] * randn(N_samples)
    else
        # log normal prior for variance terms
        prior[:, i] = exp.(log(μ_θ[i]) .+ σ_θ[i] * randn(N_samples))
    end
end

After drawing samples from our prior, the covariance matrix captures the relative uncertainty in the parameters.  Higher values of the variance mean a more uncertain parameter.

In [93]:
cov_prior = cov(prior; dims=1)

13×13 Array{Float64,2}:
 889.349       -2.97196   -12.1252     …   0.218716     0.185449  
  -2.97196    911.921      -4.40399       -0.497496    -0.49234   
 -12.1252      -4.40399   896.29          -0.0910778    0.346473  
 -10.7395      -8.39495     3.4048        -0.260282     0.0218153 
   4.12157     14.2729     10.3347        -0.0470188    0.630127  
  -5.2368       4.44168     0.779624   …  -0.323813     0.184019  
   1.5815      13.1743    -11.6491        -0.271295     0.513559  
 -12.3392      -2.37339     9.46388       -0.122911    -0.0832052 
   7.93354      1.27571   -11.762         -0.0577343   -0.174346  
  -8.78232     12.3374      4.76035       -0.666375    -0.047597  
   0.0456187   -0.407741   -0.0458951  …  -0.00828818  -0.0091964 
   0.218716    -0.497496   -0.0910778      1.03934      0.00104094
   0.185449    -0.49234     0.346473       0.00104094   1.01023   

As to be expected from the specified prior, the square root of the diagonal terms represent the original scale parameters of the prior distributions.

In [94]:
sqrt.(diag(cov_prior))

13-element Array{Float64,1}:
 29.821960011246986 
 30.198023310270607 
 29.938099249901477 
 30.287773737513852 
 30.133947276968648 
 30.157517702425746 
 29.687303803111924 
 29.708961660355943 
 29.977099611950525 
 30.01822511999994  
  1.0141675397549579
  1.019479140818277 
  1.0051009011228227

To calculate the information, we only need the total variance from the prior.

In [107]:
v = dropdims(sum(prior[:, end - 2:end], dims=2), dims=2)
histogram(v, label="", xlabel="Prior total variance");

We have a small set of cases so let's look at them.

In [96]:
cases = [
    [[1, 1], [1, 0]],
    [[1, 1], [0, 1]],
    [[1, -1], [1, 0]],
    [[1, -1], [0, 1]],
    [[-1, 1], [1, 0]],
    [[-1, 1], [0, 1]],
    [[-1, -1], [1, 0]],
    [[-1, -1], [0, 1]]
]

8-element Array{Array{Array{Int64,1},1},1}:
 [[1, 1], [1, 0]]  
 [[1, 1], [0, 1]]  
 [[1, -1], [1, 0]] 
 [[1, -1], [0, 1]] 
 [[-1, 1], [1, 0]] 
 [[-1, 1], [0, 1]] 
 [[-1, -1], [1, 0]]
 [[-1, -1], [0, 1]]

In [97]:
I = map(i -> fisher(v, t, cases[i][1], cases[i][2]), 1:length(cases));
traces = map(j -> map(i -> tr(I[j][i, :, :] * cov_prior), 1:length(v)), 1:length(cases));

In [109]:
plot()
for i in 1:length(cases)
    h = normalize(fit(Histogram, traces[i], nbins=20), mode=:pdf)
    plot!(h.edges[1][2:end], h.weights, label="Case $i", lw=2)
end
plot!(xlabel="Trace(I)", label="");

For each of the lines in the figure above is a PDF of the trace of the normalized information over the prior distribution for a given case of $x_\mathrm{bm}$ and $x_\mathrm{tx}$.  Since the information is only dependent on the prior variance and the prior covariance is the same for all fixed effects, none of the cases gives any more information than any other.

### A more informative prior

Now let's assume a set of wide zero-centered priors for all effect size parameters except for one.

In [101]:
mean_β = 0.0
scale_β = 30.0
mean_σ2 = 10.0
scale_σ2 = 0.1

μ_θ = cat(mean_β * ones(N_parameters - 3), mean_σ2 * ones(3), dims=1)
σ_θ = cat(scale_β * ones(N_parameters - 3), scale_σ2 * ones(3), dims=1)

# set an informative prior over the interaction between bm 1 and tx 1
μ_θ[7] = 10.0
σ_θ[7] = 15.0

N_samples = 10000

prior = Array{Float64, 2}(undef, N_samples, N_parameters)
for i in 1:N_parameters
    if i < N_parameters - 2
        # normal prior for effect size terms
        prior[:, i] = μ_θ[i] .+ σ_θ[i] * randn(N_samples)
    else
        # log normal prior for variance terms
        prior[:, i] = exp.(log(μ_θ[i]) .+ σ_θ[i] * randn(N_samples))
    end
end

Note that now the prior effect size for the first interaction term (the 7th index starting at 1) has significantly narrower distribution.

In [102]:
cov_prior = cov(prior; dims=1)
sqrt.(diag(cov_prior))

13-element Array{Float64,1}:
 30.425887191817324
 29.874045590301595
 30.237410191363423
 30.085890957833623
 30.22872057583287 
 29.700032673358397
 15.076636841400346
 30.08776316097902 
 30.267595539867393
 29.647429816759566
  1.012546203163566
  1.016487239148962
  0.989973456469391

The variance is the same as before, so we'll have the same information matrix.

In [103]:
v = dropdims(sum(prior[:, end - 2:end], dims=2), dims=2)
I = map(i -> fisher(v, t, cases[i][1], cases[i][2]), 1:length(cases))
traces = map(j -> map(i -> tr(I[j][i, :, :] * cov_prior), 1:length(v)), 1:length(cases));

In [108]:
plot()
for i in 1:length(cases)
    x_bm, x_tx = cases[i]
    mean_trace = round(mean(traces[i]), digits=2)
    println("Case $i : x_bm = $x_bm, x_tx = $x_tx, <trace> = $mean_trace")
    h = normalize(fit(Histogram, traces[i], nbins=20), mode=:pdf)
    plot!(h.edges[1][2:end], h.weights, label="Case $i", lw=2)
end
plot!(xlabel="Trace(I)", label="");

Case 1 : x_bm = [1, 1], x_tx = [1, 0], <trace> = 190.41
Case 2 : x_bm = [1, 1], x_tx = [0, 1], <trace> = 213.08
Case 3 : x_bm = [1, -1], x_tx = [1, 0], <trace> = 190.55
Case 4 : x_bm = [1, -1], x_tx = [0, 1], <trace> = 213.32
Case 5 : x_bm = [-1, 1], x_tx = [1, 0], <trace> = 185.65
Case 6 : x_bm = [-1, 1], x_tx = [0, 1], <trace> = 209.03
Case 7 : x_bm = [-1, -1], x_tx = [1, 0], <trace> = 190.3
Case 8 : x_bm = [-1, -1], x_tx = [0, 1], <trace> = 210.88


Here we see that four cases (2, 4, 6, 8) provide more relative information than the others.  These correspond exactly to the cases involving treatment 2 rather than treatment 1.  Since we already have more information on treatment 1 from the narrower prior distribution on the interaction effect between treatment 1 and biomarker 1, any cases with a different treatment provide more relative information.